# Exploration des niveaux de bruit complexes

Ce notebook génère un petit jeu de spectres synthétiques à l'aide du moteur physique interne et applique différents profils de bruit disponibles dans `project.data.noise`. Chaque type est exploré sur 20 niveaux (de 0.0 à 2.0) et chaque figure superpose cinq spectres pour comparer l'effet du niveau de bruit.


## Préparation

On configure l'environnement Python, on ajoute le dépôt au `sys.path` et on charge les bibliothèques utiles pour la visualisation. Un style graphique homogène est également fixé pour l'ensemble des figures.


In [ ]:
import sys
from pathlib import Path

import numpy as np
import torch
import matplotlib.pyplot as plt

from IPython.display import Markdown, display

PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / 'project').is_dir():
    for candidate in PROJECT_ROOT.parents:
        if (candidate / 'project').is_dir():
            PROJECT_ROOT = candidate
            break
    else:
        raise RuntimeError("Impossible de localiser le répertoire racine du projet.")

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.dpi'] = 110


## Génération des spectres propres

On s'appuie sur les utilitaires du projet pour charger les plages de paramètres, les transitions moléculaires de démonstration et le moteur physique `batch_physics_forward_multimol_vgrid`.

Les fichiers QTpy ne sont pas nécessaires ici : on remplace le lecteur par un stub qui retourne une partition unitaire, ce qui suffit pour visualiser l'impact du bruit.


In [ ]:
from project.config.data_config import load_parameter_ranges, load_transitions
from project.config.params import PARAMS
from project.data.noise import add_noise_variety
from project.physics.forward import batch_physics_forward_multimol_vgrid
from project.utils.lowess import lowess_value

class DummyTips:
    'Approximation minimale de Tips2021QTpy retournant Q(T)=1.'

    def __init__(self, device='cpu'):
        self.device = torch.device(device)

    def q_torch(self, mid, iso, T):
        return torch.ones_like(T, dtype=torch.float64, device=self.device)

    def q_scalar(self, mid, iso, T):
        return 1.0

PARAM_CONFIG = PROJECT_ROOT / 'project' / 'config' / 'data' / 'parameters_default.yaml'
TRANSITIONS_CONFIG = PROJECT_ROOT / 'project' / 'config' / 'data' / 'transitions_sample.yaml'
NUM_POINTS = 1024
NUM_SPECTRA = 5
MAX_RELATIVE_NOISE = 0.35

PARAM_RANGES = load_parameter_ranges(PARAM_CONFIG, update_globals=False)
TRANSITIONS_DICT, POLY_FREQ_MAP = load_transitions(TRANSITIONS_CONFIG, include_poly_freq=True)
POLY_FREQ_CH4 = POLY_FREQ_MAP.get('CH4')
TIPSPY = DummyTips()

def generate_reference_batch(n_samples=NUM_SPECTRA, seed=42):
    torch.manual_seed(int(seed))
    params = {
        name: torch.empty(n_samples, dtype=torch.float32).uniform_(*PARAM_RANGES[name])
        for name in PARAMS
    }

    baseline_coeffs = torch.stack([params['baseline0'], params['baseline1'], params['baseline2']], dim=1)
    v_grid_idx = torch.arange(NUM_POINTS, dtype=torch.float32)

    mf_dict = {}
    if TRANSITIONS_DICT.get('CH4'):
        mf_dict['CH4'] = params['mf_CH4']
    if TRANSITIONS_DICT.get('H2O'):
        mf_dict['H2O'] = params['mf_H2O']

    clean, v_grid = batch_physics_forward_multimol_vgrid(
        params['sig0'],
        params['dsig'],
        POLY_FREQ_CH4,
        v_grid_idx,
        baseline_coeffs,
        TRANSITIONS_DICT,
        params['P'],
        params['T'],
        mf_dict,
        tipspy=TIPSPY,
        device='cpu',
    )

    clean = clean.to(torch.float32)
    baseline = lowess_value(clean, kind='start', win=30).to(torch.float32)

    return {
        'clean': clean,
        'baseline': baseline,
        'frequency': v_grid[0].to(torch.float32),
    }

def apply_noise_profile(clean, baseline, noise_type, noise_level, seed):
    g = torch.Generator(device='cpu')
    g.manual_seed(int(seed))
    profile = {
        'legacy_enabled': False,
        'complex': {
            'mode': 'replace',
            'noise_type': str(noise_type),
            'noise_level': float(noise_level),
            'max_rel_to_line': MAX_RELATIVE_NOISE,
            'clip': (0.0, 1.5),
        },
    }
    return add_noise_variety(clean, generator=g, baseline_norm=baseline, **profile)


In [ ]:
reference = generate_reference_batch(n_samples=NUM_SPECTRA, seed=123)
clean_spectra = reference['clean']
baseline = reference['baseline']
frequency = reference['frequency']

freq_np = frequency.cpu().numpy()
clean_np = clean_spectra.cpu().numpy()

fig, axes = plt.subplots(NUM_SPECTRA, 1, sharex=True, figsize=(12, 2.2 * NUM_SPECTRA))
axes = np.atleast_1d(axes)
for idx, ax in enumerate(axes):
    ax.plot(freq_np, clean_np[idx], color='black')
    ax.set_ylabel(f'#{idx + 1}')
axes[-1].set_xlabel('Fréquence (cm⁻¹)')
fig.suptitle('Spectres propres utilisés pour la comparaison')
fig.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()


## Balayage des niveaux de bruit

Chaque type de bruit est balayé sur 20 niveaux également espacés entre 0.0 et 2.0. Les cinq spectres de référence sont conservés, seuls les tirages de bruit changent (graine différente à chaque figure).


In [ ]:
noise_types = ['gaussian', 'shot', 'flicker', 'etaloning', 'glitches']
noise_levels = np.linspace(0.0, 2.0, 20)
color_map = plt.cm.plasma(np.linspace(0.2, 0.85, NUM_SPECTRA))

for noise_type in noise_types:
    display(Markdown("### Profil : `{}`".format(noise_type)))
    for idx, noise_level in enumerate(noise_levels, start=1):
        noisy = apply_noise_profile(clean_spectra, baseline, noise_type, noise_level, seed=10000 + idx)
        noisy_np = noisy.cpu().numpy()

        fig, axes = plt.subplots(NUM_SPECTRA, 1, sharex=True, figsize=(12, 2.2 * NUM_SPECTRA))
        axes = np.atleast_1d(axes)
        for sp_idx, ax in enumerate(axes):
            label_clean = 'Propre' if sp_idx == 0 else None
            label_noisy = 'Bruité' if sp_idx == 0 else None
            ax.plot(freq_np, clean_np[sp_idx], linestyle='--', color='black', alpha=0.65, label=label_clean)
            ax.plot(freq_np, noisy_np[sp_idx], color=color_map[sp_idx], label=label_noisy)
            ax.set_ylabel(f'#{sp_idx + 1}')

        axes[0].legend(loc='upper right')
        axes[-1].set_xlabel('Fréquence (cm⁻¹)')
        fig.suptitle('{} - noise_level = {:.2f}'.format(noise_type.capitalize(), noise_level))
        fig.tight_layout(rect=[0, 0, 1, 0.97])
        plt.show()
